In [1]:
import numpy as np
import pandas as pd

## Gene co-expression network

A gene co-expression network (GCN) is an undirected graph, where each node corresponds to a gene, and a pair of nodes is connected with an edge if there is a significant co-expression relationship between them.

In this context we want to link only those genes that are directly dependent from one another and not linked through a third gene. 

In the state of the art you may find several methods for the inference of graphs based on different concepts. Today we are going to see **Algorithm for the Reconstruction of Accurate Cellular Networks** (ARACNe)

### Loading data
Load the files we used last lab `"gedm.csv"` and `"labels.csv"`.

In [2]:
data = pd.read_csv("gedm.csv", index_col=0)
labels = pd.read_csv("labels.csv", index_col=0)

#### Remove rows that start with AFFX and select a sub-matrix

In [3]:
data.drop(data.index[data.index.str.startswith('AFFX')], inplace=True)
X = data.iloc[np.random.randint(0, data.shape[0], 30), :].values.T

# ARACNe

https://bmcbioinformatics.biomedcentral.com/articles/10.1186/1471-2105-7-S1-S7

It is a method for the reverse engineering of transcriptional networks. It defines an edge between two gene expression profiles as an *irreducible statistical dependency* that cannot be explained as an artifact or other statistical dependencies in the network.

The joint probability distribution over the genes is defined as 
<img src="jpd.png"/>
where $Z$ is a normalization factor, $N$ is the number of genes and $\Phi$ are the potentials. In the model we say that two variables interact iff their potential is nonzero. 

The estimation of the network is done through 3 main steps:

    1) computation of pairwise mutual information
    2) statistical threshold for mutual information
    3) data processing inequality

## Computation of pairwise mutual information
The mutual information $I(g_i, g_j) = I_{ij}$ is an information-theoretic measure of relatedness that is zero iff $P(g_i, g_j) = P(g_i)P(g_j)$ i.e., if the genes $g_i$ and $g_j$ are independent. 

The mutual information is defined, for a pair of random variables $x$ and $y$ as $I(x,y) = S(x) + S(y) - S(x,y)$ where $S(t)$ is the *entropy*.

The entropy of a random variable t is defined as: 


<center>$\Large{S(t) = - \mathbb{E}[\text{log}p(t)] = - \sum\limits_{i}p(t_i)\text{log}(p(t_i))}$</center>

with $p(t_i)$ is the probability of each discrete state of the variable.

Therefore
<center>$\Large{I(g_i,g_j)=I_{ij}=\sum\limits_{h,k}p(g_{ih},g_{jk})log\bigg{[}\frac{p(g_{ih},g_{jk})}{p(g_ih)p(g_jk)}\bigg{]}}$</center>.

This measure can be generalized to the continuous case easily. 
Mutual information is guaranteed to be nonzero iff any kind of statistical dependence exists. 

Above definitions are well defined for discrete data and probability mass functions. When dealing with **continuous random variables** (as *microarray data*) we need to realiably estimate the density function from data in order to be able to compute the mutual information between each pair of genes.

**Kernel Density Estimation (KDE)** is a nonparametric method for estimating probability densities.

We can approximate the underlying density of our data by computing,

<center>$\Large{\hat{p}(x)=\frac{1}{n}\sum\limits_{i}^H\mathbf{K}(u)}$,</center>

where $\large{u=\frac{(x-x_i)^TS^{-1}(x-x_i)}{h^2}}$, $\mathbf{K}(u)$ is a multivariate kernel function, $x=[x_1,\dots,x_d]^T$ is the $d$-dimensional random vector whose density is being estimated, $x_i=[x_{i1},\dots,x_{in}]^T$ with $i=1,\dots,n$ are the n sample vectors, $h$ is the kernel bandwidth, $S$ is the covarinace matrix of the observations $x_i$ and $H$ prescribes the range of data over which the average is computed.

We use the multivariate Gaussian probability density function for the kernel function $\mathbf{K}(u)$ which is given by 


<center>$\large{\mathbf{K}(u)=\frac{1}{(2\pi)^{\frac{d}{2}}h^d det(S)^{\frac{1}{2}}}exp(-\frac{u}{2})}$</center>

**In `utils.py` you may find already defined functions for the kernel density estimation of the joint probability denisty function (`p_mkde_M`), the marginals (`p_kde`) and for the approximation of mutual information using kernel estimations (`kernel_mi`).**

**Use these functions to compute the $N_{genes}\times N_{genes}$ mutual information matrix (MI) from your data**

In [4]:
from utils import kernel_mi
import matplotlib.pyplot as plt

%matplotlib inline

In [5]:
def compute_mutual_information(X):
    #Insert your code here
    return MI

MI = compute_mutual_information(X)

NameError: name 'MI' is not defined

**Visualize the Gene expressions and the mutual information**

In [ ]:
plt.figure(figsize=(13, 13))

plt.subplot(121)
plt.imshow(X,interpolation='None')
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('Gene expression')

plt.subplot(122)
plt.imshow(MI,interpolation='None')
plt.colorbar(fraction=0.046, pad=0.04)
plt.title('Mutual Information between gene pairs');

## Statistical threshold for mutual information

Since MI is always non-negative its evaluation gives positive values even for variables that are mutually independent. Therefore we need to eliminate those edges for which there is no evidence against the hypothesis of mutual independence (the null hypothesis).

We want to obtain a statistical threshold $I_0$ on computed mutual informations able to identify those connections that do not actually exist.
To this extent, fixed a p-value, consider a set of genes and a **bilateral statistical test** of samples independence.

We use as test statistic the mutual information between pairs of genes. In order to compute the p-value we should know the distribution of mutual informations under the null-hypothesis, since the p-value is defined as

<center>$\large{p_{H_0}(I>=I_0)}$.</center>

Nevertheless, we do not know the distribution of mutual information on independent variables. We need to perform **Monte Carlo** simulations on independent data in order to obtain an approximate distribution of MIs. We do this by shuffling the genes across the profiles and evaluate the MI of these manifestly independent genes.


We then compute the threshold based on our confidence.

**Visualize the distribution of observed mutual informations on our data**

In [ ]:
aux = np.ndarray.flatten(MI)
idxx = np.where(aux != 0)
MI_list = np.ndarray.tolist(aux[idxx])

#Plot
plt.figure(figsize=(10, 6))
bins = np.linspace(min(MI_list), max(MI_list), 101)
plt.hist(MI_list, bins, facecolor='b',alpha=0.75)
plt.xlabel('Mutual Information')
plt.ylabel('Count of associations')
plt.title('Distribution of observed Mutual Information');

**Fill the function below aimed to compute the mutual information on different permutations of data**

In [ ]:
def permutated_MI(X, N_perm):
    
    #Insert your code here
        
    return MI_perm

In [ ]:
MI_perm = permutated_MI(X, N_perm=1) 
MI_perm = np.ndarray.flatten(MI_perm)
idx = np.where(MI_perm != 0)
MI_list2 = np.ndarray.tolist(MI_perm[idx])

**Visualize the data empirical distribution together with the approximated empirical distribution of independent samples. What can you observe?**

In [ ]:
#Plot
plt.figure(figsize=(10, 6))

plt.hist(MI_list, bins,facecolor='b', alpha=0.75,label='original data')
plt.hist(MI_list2, bins,facecolor='g', alpha=0.75,label='shuffled data')
plt.legend(loc='upper right')

# plt.axvline(np.amax(matMI_alternative),c='r',linewidth=4)
plt.xlabel('Mutual Information')
plt.ylabel('Count of associations')
plt.title('Distribution of calculated Mutual Information');

**Let's compute the empirical complementary cumulative distribution function of the synthetic data
and compute the threshold based on a fixed significance level p**

In [ ]:
p_value = 0.1

def eccd(synth_data,data):
    #Insert your code here
    return emp_eccd

**Visualize the empirical complememtary cumulative distribution function on data defined below. Is it coherent with what you expected?**

In [ ]:
data = #?
plt.figure(figsize=(10,6))
plt.plot(data,eccd(MI_perm,data),lw = 2,color='b')

**We compute the statistical threshold by looking for the value of the synthetic data for which the probability
of observing values greater than that is less or equal than p**

In [ ]:
data = np.unique(np.sort(MI_perm))
observed_eccd = eccd(MI_perm,data)

Compute the statistical threshold $I_0$

In [ ]:
print('Statistical threshold: $I_0$')

**Visualize the data empirical distribution together with the approximated empirical distribution of independent sample and the estimated statistical threshold. What can you obseerve?**

In [ ]:
#Plot
plt.figure(figsize=(10, 6))

plt.hist(MI_list, bins,facecolor='b', alpha=0.75,label='original data')
plt.hist(MI_list2, bins,facecolor='g', alpha=0.75,label='shuffled data')
plt.legend(loc='upper right')

plt.axvline(I_0,c='r',linewidth=4)
plt.xlabel('Mutual Information')
plt.ylabel('Count of associations')
plt.title('Distribution of calculated Mutual Information');

print('Maximum MI value for the shuffled matrix: {}'.format(np.max(MI_perm)))

**Use `plt.imshow` to visualize the MI matrix with and without sparsity induced by the threshold** 

In [ ]:
MI_filtered = #The mutual information matrix with edges removed cause of the sparisty induced by the statistical threshold


#Plot
plt.figure(figsize=(13, 13))


## Data processing inequality (DPI)

We want to exclude those genes that are connected through a third gene. We now that if the genes $g_1$ and $g_3$ are connected through $g_2$ then

$I(g_1, g_3) \leq \text{min}\bigg(I(g_1, g_2), I(g_2, g_3)\bigg)$

therefore we look at all the triplets of genes in the resulting matrix of the previous step and we eliminate those edges that respects this inequality. 

**Define a function able to detect the smallest mutual information between a triplets and remove the corresponding edge in the final graph**

In [ ]:
def filter_cycles(MI_filtered):
    
    return MI_final

**Use `plt.imshow` to visualize the MI matrices with and without sparsity induced by the threshold and DPI** 

In [ ]:
#Plot

# Synthetic experiment

**We evaluate the previous pipeline for the inference of regulatory networks from gene expression data comparing the inferred network with a 'syntehtic' one. The synthetic one is obtained by simulating sort of metabolic processes, based on sets of coupled differential equations. To fully define a gene network model it is also necessary to create a network topology, or wiring diagram. Past research in gene networks has concentrated on a particular topological class: random gene networks. These random networks follow a topology studied earlier by mathematicians Erdo ̈s and Re ́nyi, where each vertex of a graph is equally likely to be connected to any other vertex in the graph.**

**We obtained a dataset compatible with the above description at the `Artificial Gene Network Series Century`[http://www.comp-sys-bio.org/AGN/Century/]**

In [ ]:
#Load the gene expression data

df = pd.read_csv('CenturyRND.csv',sep='\t')
df = pd.DataFrame(df)
df.columns = ['G'+str(i) for i in range(1,101)]

In [ ]:
df.head()

**Load the network edges**

In [ ]:
edges = pd.read_csv('CenturyRND-net.csv', sep=' ')
edges = pd.DataFrame(edges)
edges = edges[['Node1','Node2']]
edges.head()

**Let's build the network with `networkx` and obtain the adjacency matrix**

In [ ]:
import networkx as nx

edges_ground_truth = edges.values
ground_net = nx.Graph()
ground_net.add_nodes_from([i for i in range(1,101) if i not in cols_to_delete])
ground_net.add_edges_from(edges_ground_truth)
adj_ground_net = nx.adjacency_matrix(ground_net).todense()

In [ ]:
plt.figure(figsize = (7,7))
nx.draw_circular(ground_net,with_labels=True, node_size=400, node_color='cyan',)

In [ ]:
plt.figure(figsize=(7,7))
plt.imshow(adj_ground_net)
plt.title("Adjacency matrix of the ground-truth network")

**Use ARACNe method for inferring the network. Refer to the previous pipeline. We have already computed the matrices MI and MI_perm since their computation is pretty expensive. You just need to load them as `np.array`**

In [ ]:
from numpy import load

MI = load('MI.npy')
MI_perm = load('MI_perm.npy')

**Define a function able to recover the undirected network from the inferred mutual information matrix and find a way to compare the obtained adjacency matrix with the ground-truth in order to evaluate the inference result.**